In [1]:
# NOTEBOOK IMPORTS
import os
import glob
import numpy as np
from tqdm.notebook import tqdm
from shutil import copyfile

# IMAGE IMPORTS
import cv2
from PIL import Image

# GIS IMPORTS
import fiona
import pyproj
from affine import Affine
from shapely.geometry import shape, mapping
from shapely.geometry import Point, LineString
from shapely.ops import transform, nearest_points, snap
import geopandas as gpd
import rasterio as rio
from rasterio.mask import mask
from scipy.spatial import cKDTree

# PLOTTING IMPORTS
import matplotlib.pyplot as plt
import matplotlib.patches as patches


# CUSTOM UTILITIES
from WorldFileUtils import *
from GeometryUtils import *
from icp import *

Image.MAX_IMAGE_PIXELS = 933120000

In [2]:
templates_dir = "data/templates/"
tempfiles_dir = "tempfiles/"

boundary_shapefile = f"{templates_dir}HCAD_Harris_County_Boundary.shp"
boundary_points    = f'{tempfiles_dir}boundary_points.shp'

In [3]:
line = fiona.open(boundary_shapefile)

firstline = line.next()
first = shape(firstline['geometry']).boundary

wgs84 = pyproj.CRS('EPSG:4326')
utm = pyproj.CRS('EPSG:3857')

project = pyproj.Transformer.from_crs(wgs84, utm, always_xy=True).transform
first = transform(project, first)

# length of the LineString
length = first.length

point_boundary_list = list()

if not os.path.exists(boundary_points):
    for distance in tqdm(range(0,int(length),5)):
        point = first.interpolate(distance)   
        point_boundary_list.append(point)
        point_boundary_gdf = gpd.GeoDataFrame(geometry=point_boundary_list)
        point_boundary_gdf.to_file(boundary_points)

C:\Users\franc\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: FionaDeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
%%timeit -n 1 -r 1

# BOUNDARY SHAPEFILE AND BUFFER
boundary_shp = gpd.read_file(boundary_shapefile).to_crs("EPSG:3857")
boundary_buf = boundary_shp.boundary.buffer(1000)

# BOUNDARY POINTS 
boundary_points_gdf_or = gpd.read_file(boundary_points)

initial_filename = f"{tempfiles_dir}border.png"
current_filename = initial_filename

plot = False

for i in range(5):
    output_filename = f"{tempfiles_dir}/out/border_{i}.png"
    
    # GET ORIGINAL TRANSFORM FROM IMAGE AND CONVERT TO AFFINE OBJECT
    original_transform, _ = get_geotransform_from_tfw(current_filename[:-3]+"pgw")
    ot = Affine.from_gdal(*original_transform)
    
    # CANDIDATE PIXEL (KEYPOINT) COORDINATES FROM TILE
    rows, cols = get_true_pixel_rc(current_filename, polygon=boundary_buf)    
    coords_gdf = gpd.GeoDataFrame(geometry=gpd.points_from_xy(rows, cols ))
    coords_gdf = coords_gdf.sample(n=1000)
                                 
    # PIXEL COORDINATES FROM CLOSEST POINT IN LINE, NORMALIZED TO INVERSE TRANSFORM OF SHAPEFILE
    point_boundary_gdf = boundary_points_gdf_or.copy()
    point_boundary_gdf = normalize_geometry(point_boundary_gdf, ot)
    boundary_line = LineString(point_boundary_gdf.geometry)
    boundary_points_matching = find_points_on_linestring(boundary_line, coords_gdf.geometry)
    
    # COORDINATE ARRAY 
    boundary_points_matching_coords = np.array([np.array([m.x, m.y]) for m in boundary_points_matching])
    
    # TO AND FROM POINTS
    to_points = np.array(boundary_points_matching_coords).astype(np.float32)
    from_points = np.array([np.array([cg.x, cg.y]).astype(np.float32) for cg in coords_gdf.geometry])
    
    # CALCULATE HOMOGRAPHY
    output = cv2.findHomography(from_points, to_points, cv2.RANSAC,1000)
    homography = output[0]
    
    # GET CURRENT TRANSFORM
    original_transform, _ = get_geotransform_from_tfw(current_filename[:-3]+"pgw")
    current_transform = getActualTransform(original_transform, homography.flatten()[:6])
    
    # OUTPUT
    copyfile(current_filename, output_filename)
    write_world_file_from_affine(current_transform, output_filename[:-3]+"pgw")
    current_filename = output_filename

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

1min 8s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
